In [1]:
import pandas as pd
import numpy as np

import plotly.offline as pl
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
import cufflinks as cf
cf.go_offline() 

import warnings
warnings.filterwarnings("ignore")

In [2]:
df=pd.read_csv('data.csv', delimiter='\t', encoding='cp1251')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 422 entries, 0 to 421
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Month        422 non-null    object
 1   Filial       422 non-null    int64 
 2   Doctor       422 non-null    object
 3   Department   422 non-null    object
 4   Clients      422 non-null    int64 
 5   New Clients  422 non-null    int64 
 6   Income       422 non-null    int64 
dtypes: int64(4), object(3)
memory usage: 23.2+ KB


In [4]:
df['Filial']=df['Filial'].astype('string')

In [5]:
df.head()

,Month,Filial,Doctor,Department,Clients,New Clients,Income
0,01.01.2019,1,A,Пародонтология,33,11,136600
1,01.01.2019,1,A,Детская стоматология,3,0,3395
2,01.01.2019,1,A,Терапия,3,1,8965
3,01.01.2019,1,A,Средства профилактики,6,0,5445
4,01.01.2019,3,B,Ортопедия,26,2,204750


In [6]:
df[['Month','Income']].groupby('Month').sum(
    ).iplot(xTitle='Месяц',
            yTitle='Сумма',
            title='Динамика поступлений',
            mode='lines+markers' ,
            theme='space', colorscale='ggplot')

In [7]:
df[['Month','Filial','Income']].groupby(['Month','Filial'], as_index=False).sum(
    ).iplot(x='Month', y='Income',
    mode='lines+markers',title='Динамика начислений по филиалам',
    categories='Filial', theme='space', colorscale='ggplot')

In [8]:
df.groupby('Filial', as_index=False).sum()
fig = make_subplots(rows=1, cols=3, specs=[[{'type':'domain'},{'type':'domain'},{'type':'domain'}]])

fig.add_trace(go.Pie(labels=df['Filial'], values=df['Income'], title='Поступления'),
              1, 1)
fig.add_trace(go.Pie(labels=df['Filial'], values=df['Clients'], title='Клиенты'),
              1, 2)
fig.add_trace(go.Pie(labels=df['Filial'], values=df['New Clients'], title='Первичные'),
              1, 3)
fig.update_traces(hole=.4, hoverinfo="label+percent+name")
fig.update_layout(title='Сравнение филиалов по фин.поступлениям, клиентам и первичным пациентам.')

fig.show()

In [9]:
df[['Department','Income','Clients']].groupby('Department').sum().sort_values(by='Income',ascending=False).iplot(
    kind='bar',
    xTitle='Отделение',
    yTitle='Количество',
    secondary_y='Clients',
    title='Структура поступлений и посещений по отделениям',
    theme='space', colorscale='plotly')

In [10]:
df1=df[['Filial','Department','Income']].loc[df['Filial']=='1'].groupby(['Filial','Department'],as_index=False).sum()
df2=df[['Filial','Department','Income']].loc[df['Filial']=='2'].groupby(['Filial','Department'],as_index=False).sum()
df3=df[['Filial','Department','Income']].loc[df['Filial']=='3'].groupby(['Filial','Department'],as_index=False).sum()

fig = make_subplots(rows=1, cols=3, specs=[[{'type':'domain'},{'type':'domain'}, {'type':'domain'}]])

fig.add_trace(go.Pie(labels=df1['Department'], values=df1['Income'], title='№1'),
              1, 1)
fig.add_trace(go.Pie(labels=df2['Department'], values=df2['Income'], title='№2'),
              1, 2)
fig.add_trace(go.Pie(labels=df3['Department'], values=df3['Income'], title='№3'),
              1, 3)
fig.update_traces(hole=.4, hoverinfo='label+percent+name')
fig.update_layout(title='Доли отделений в каждом из филиалов')
fig.show()

In [11]:
df[['Department','Clients','New Clients']].groupby('Department').sum().sort_values(by='Clients',ascending=False).iplot(
    kind='bar',
    xTitle='Отделение',
    yTitle='Количество',
    title='Поток пациентов',
    theme='space', colorscale='plotly')

In [12]:
df[['Doctor','Income','Clients']].groupby('Doctor'
    ).sum().sort_values('Income',ascending=False).iplot(
    kind = 'bar', title='Сравнение количества клиентов и дохода с разбивкой по доктору',
    secondary_y='Clients', secondary_y_title='Клиентов',
    theme ='space', colorscale='ggplot')

In [13]:
a=df[['Department','Doctor','Income']].loc[df['Department']!='Средства профилактики'].groupby(['Department','Doctor'],as_index=False).sum()
fig=px.sunburst(a,path=['Department','Doctor'],values='Income')
fig.update_layout(title='Лучевая диаграмма долей отделений и докторов в поступлениях организации')
fig.show()

In [14]:
df.groupby(['Doctor'],as_index=False).sum().iplot(kind='scatter3d',
        x='Clients', xTitle='Клиентов',
        y='New Clients', yTitle='Новых клиентов', 
        z='Income', zTitle='Сумма', title='3D диаграмма сравнения докторов',
        categories='Doctor' ,colorscale='plotly')